<a href="https://colab.research.google.com/github/luiis11/tesis-lcc/blob/main/Notebooks/BinaryClass_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Library Imports

In [ ]:
# General Libraries
%matplotlib inline
import pandas as pd
import numpy as np
import os               # para manipulacion de directorios
import shutil           # para eliminar carpetas no vacias
!pip install mglearn    # el comando 'import mglearn' no funcionaba

# Preprocessing
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords

# CountVectorizer | TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

# Cross Validation
from sklearn.model_selection import cross_val_score

# GridSearch
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

# LogisticRegression Classifier
from sklearn.linear_model import LogisticRegression

# SVM Classifier
from sklearn import svm

# KNN Clasificator
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler

# Gaussian NaiveBayes Clasificator
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB

# RandomForest
from sklearn.ensemble import RandomForestClassifier

# DecisionTree (Classifier and Regressor)
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor

# Graphics library
import matplotlib.pyplot as plt

### Read CSV
**Selección del la personalidad**

Se utiliza un valor dentro del rango de las personalidades para entrenar el clasificador binario en la personalidad seleccionada.
Los cuales tienen un total de 225 filas y cuentan con la siguiente estructura en columnas:

*   ID: Number
*   Personality: ['si', 'no']
*   Autodescripción: String

In [ ]:
# Select the Personality

# 0: Agrado
# 1: Apertura
# 2: Extraversion
# 3: Neuroticismo
# 4: Responsabilidad

POS = 4

personality = ["Agrado", "Apertura", "Extraversion", "Neuroticismo", "Responsabilidad"]
colum_title = ["AgradPerc", "ApertPerc", "ExtravPerc", "NeurPerc", "RespPerc"]

path_colab_original = [
    "https://drive.google.com/file/d/138orceXr83_CdJiVvC7bb9IF4-J8P6fp/view", #"./Agrado_Full.csv",
    "https://drive.google.com/file/d/1598rz27p2WRDtq6W1kQF1Pp0uigzpB_U/view", #"./Apertura_Full.csv" ,
    "https://drive.google.com/file/d/187RaPP8EnCjetblpr6YuHuCIXuASVDoa/view", #"./Extraversion_Full.csv" ,
    "https://drive.google.com/file/d/18h2J-jBaPXbhROjLnglukvYaF_Mv4BmN/view", #"./Neuroticismo_Full.csv" ,
    "https://drive.google.com/file/d/1wr20m6Cr7HL91rnbM3t5tfDr5GTZcHpS/view", #"./#Responsabilidad_Full.csv"
]

path_colab_augment  = [
    "https://drive.google.com/file/d/1MvPCzl_1C7LRvo9Yev10e-46qWiI-D08/view", #"./Augment_gpt_Agrado.csv",
    "https://drive.google.com/file/d/1VSQl39Q1pU9zmpkAjKbi2ZhB96_qpe1r/view", #"./Augment_gpt_Apertura.csv",
    "https://drive.google.com/file/d/19vFX82WeQuB2_TXTw4VUUrl3iqYM54E2/view", #"./Augment_gpt_Extraversion.csv",
    "https://drive.google.com/file/d/1JB-g343XOFtDNIiuK1jV1aOfzcLNY_w1/view", #"./Augment_gpt_Neuroticismo.csv",
    "https://drive.google.com/file/d/1WeA1BUlsGWYvC6HUm5lO01XQ8d09ssJZ/view", #"./Augmented_gpt_RespPerc.csv"
]

path_colab = path_colab_augment # Select the augment dataset


# Set values
path_colab = path_colab[POS]
colum_labels = colum_title[POS]

# With ONLY URL
if path_colab.startswith('http'):
  print("Ubication WEB")
  path_colab='https://drive.google.com/uc?id=' + path_colab.split('/')[-2]
else: print("Ubication Local")


Ubication WEB


In [ ]:
# Importar un CSV con pandas a un formato DataFrame (df)

df = pd.read_csv(
    path_colab,
    encoding = "utf-8",
    header = 0
)

texts_unprocessed = list(df["Autodescripción"]) # será el X_train o simplemente X
labels = list(df[colum_labels])                 # será el y_train o simplemente y
print("Textos:   ", len(texts_unprocessed))
print("Etiquetas:", len(labels))

#df = df.drop(df.tail(24).index) # Remover ultimas n filas

print(f"+({df[colum_labels].sum()})")
print(f"-({len(df)-df[colum_labels].sum()})")

Textos:    470
Etiquetas: 470
+(284)
-(186)


### Preprocesamiento
**Preprocesamiento del texto (opcional)**

Es posible realizar una limpieza del texto, quitando carácteres innecesarios como números, símbolos, e incluso palabras que no aportan significancia a la oración.

Antes de aplicar la funcion CountVectorizer o TfidfVectorizer (es decir, obtener la representación), es posible realizar un preprocesamiento del texto de entrada para ´limpiarlo´ un poco.

En este caso, luego de leer el corpus se procede a preprocesar el texto de la columna Autodescripción con las siguientes técnicas.

In [ ]:
def preprocessing(texto):
    texto = texto.replace('.', '. ').replace(',', ', ').replace(';', '; ').replace('(', '( ').replace(')', ') ') # Separar palabras antes de quitar las comas y puntos para evitar que palabras queden juntas
    texto = re.sub(r'[^\w ]', "", texto)    # Eliminar todo lo que no sea una palabra
    texto = re.sub(r'\d+', 'numero', texto)   # Cambiar numeros por la palabra numero
    texto = texto.lower()                      # Pasa el texto a minúscula
    letras_dobles = "abdfghijkmnñpqstuvwxyz" # Remover letras con 2 ocurrencias (con excepciones). Excepciones: ee-cc-ll-rr-oo (y mayus)
    letras_dobles += letras_dobles.upper()
    texto = re.sub("(?P<char>[" + re.escape(letras_dobles) + "])(?P=char)+", r"\1", texto)
    texto = re.sub(r'([\w\W])\1{2,}', r'\1', texto)  # remover caracteres que se repiten al menos 3 veces
    return texto

texts_processed = []

for i in texts_unprocessed:
    x = preprocessing(i)
    texts_processed.append(x)

# Selección de Texto original o preprocesado (Comentar el que no se utiliza)

# texts = texts_unprocessed
texts = texts_processed

In [ ]:
# Utilizar stop_words extendido

# stopwords_sp_web = ["algún","alguna","algunas","alguno","algunos","ambos","ampleamos","ante","antes","aquel","aquellas","aquellos","aqui","arriba","atras","bajo","bastante","bien","cada","cierta","ciertas","cierto","ciertos","como","con","conseguimos","conseguir","consigo","consigue","consiguen","consigues","cual","cuando","dentro","desde","donde","dos","el","ellas","ellos","empleais","emplean","emplear","empleas","empleo","en","encima","entonces","entre","era","eramos","eran","eras","eres","es","esta","estaba","estado","estais","estamos","estan","estoy","fin","fue","fueron","fui","fuimos","gueno","ha","hace","haceis","hacemos","hacen","hacer","haces","hago","incluso","intenta","intentais","intentamos","intentan","intentar","intentas","intento","ir","la","largo","las","lo","los","mientras","mio","modo","muchos","muy","nos","nosotros","otro","para","pero","podeis","podemos","poder","podria","podriais","podriamos","podrian","podrias","por","por qué","porque","primero","puede","pueden","puedo","quien","sabe","sabeis","sabemos","saben","saber","sabes","ser","si","siendo","sin","sobre","sois","solamente","solo","somos","soy","su","sus","también","teneis","tenemos","tener","tengo","tiempo","tiene","tienen","todo","trabaja","trabajais","trabajamos","trabajan","trabajar","trabajas","trabajo","tras","tuyo","ultimo","un","una","unas","uno","unos","usa","usais","usamos","usan","usar","usas","uso","va","vais","valor","vamos","van","vaya","verdad","verdadera","verdadero","vosotras","vosotros","voy","yo","él","ésta","éstas","éste","éstos","última","últimas","último","últimos","a","añadió","aún","actualmente","adelante","además","afirmó","agregó","ahí","ahora","al","algo","alrededor","anterior","apenas","aproximadamente","aquí","así","aseguró","aunque","ayer","buen","buena","buenas","bueno","buenos","cómo","casi","cerca","cinco","comentó","conocer","consideró","considera","contra","cosas","creo","cuales","cualquier","cuanto","cuatro","cuenta","da","dado","dan","dar","de","debe","deben","debido","decir","dejó","del","demás","después","dice","dicen","dicho","dieron","diferente","diferentes","dijeron","dijo","dio","durante","e","ejemplo","ella","ello","embargo","encuentra","esa","esas","ese","eso","esos","está","están","estaban","estar","estará","estas","este","esto","estos","estuvo","ex","existe","existen","explicó","expresó","fuera","gran","grandes","había","habían","haber","habrá","hacerlo","hacia","haciendo","han","hasta","hay","haya","he","hecho","hemos","hicieron","hizo","hoy","hubo","igual","indicó","informó","junto","lado","le","les","llegó","lleva","llevar","luego","lugar","más","manera","manifestó","mayor","me","mediante","mejor","mencionó","menos","mi","misma","mismas","mismo","mismos","momento","mucha","muchas","mucho","nada","nadie","ni","ningún","ninguna","ningunas","ninguno","ningunos","no","nosotras","nuestra","nuestras","nuestro","nuestros","nueva","nuevas","nuevo","nuevos","nunca","o","ocho","otra","otras","otros","parece","parte","partir","pasada","pasado","pesar","poca","pocas","poco","pocos","podrá","podrán","podría","podrían","poner","posible","próximo","próximos","primer","primera","primeros","principalmente","propia","propias","propio","propios","pudo","pueda","pues","qué","que","quedó","queremos","quién","quienes","quiere","realizó","realizado","realizar","respecto","sí","sólo","se","señaló","sea","sean","según","segunda","segundo","seis","será","serán","sería","sido","siempre","siete","sigue","siguiente","sino","sola","solas","solos","son","tal","tampoco","tan","tanto","tenía","tendrá","tendrán","tenga","tenido","tercera","toda","todas","todavía","todos","total","trata","través","tres","tuvo","usted","varias","varios","veces","ver","vez","y","ya"]
stopwords_sp_web_accent_less = set({"algun","alguna","algunas","alguno","algunos","ambos","ampleamos","ante","antes","aquel","aquellas","aquellos","aqui","arriba","atras","bajo","bastante","bien","cada","cierta","ciertas","cierto","ciertos","como","con","conseguimos","conseguir","consigo","consigue","consiguen","consigues","cual","cuando","dentro","desde","donde","dos","el","ellas","ellos","empleais","emplean","emplear","empleas","empleo","en","encima","entonces","entre","era","eramos","eran","eras","eres","es","esta","estaba","estado","estais","estamos","estan","estoy","fin","fue","fueron","fui","fuimos","gueno","ha","hace","haceis","hacemos","hacen","hacer","haces","hago","incluso","intenta","intentais","intentamos","intentan","intentar","intentas","intento","ir","la","largo","las","lo","los","mientras","mio","modo","muchos","muy","nos","nosotros","otro","para","pero","podeis","podemos","poder","podria","podriais","podriamos","podrian","podrias","por","por que","porque","primero","puede","pueden","puedo","quien","sabe","sabeis","sabemos","saben","saber","sabes","ser","si","siendo","sin","sobre","sois","solamente","solo","somos","soy","su","sus","tambien","teneis","tenemos","tener","tengo","tiempo","tiene","tienen","todo","trabaja","trabajais","trabajamos","trabajan","trabajar","trabajas","trabajo","tras","tuyo","ultimo","un","una","unas","uno","unos","usa","usais","usamos","usan","usar","usas","uso","va","vais","valor","vamos","van","vaya","verdad","verdadera","verdadero","vosotras","vosotros","voy","yo","el","esta","estas","este","estos","ultima","ultimas","ultimo","ultimos","a","anadio","aun","actualmente","adelante","ademas","afirmo","agrego","ahi","ahora","al","algo","alrededor","anterior","apenas","aproximadamente","aqui","asi","aseguro","aunque","ayer","buen","buena","buenas","bueno","buenos","como","casi","cerca","cinco","comento","conocer","considero","considera","contra","cosas","creo","cuales","cualquier","cuanto","cuatro","cuenta","da","dado","dan","dar","de","debe","deben","debido","decir","dejo","del","demas","despues","dice","dicen","dicho","dieron","diferente","diferentes","dijeron","dijo","dio","durante","e","ejemplo","ella","ello","embargo","encuentra","esa","esas","ese","eso","esos","esta","estan","estaban","estar","estara","estas","este","esto","estos","estuvo","ex","existe","existen","explico","expreso","fuera","gran","grandes","habia","habian","haber","habra","hacerlo","hacia","haciendo","han","hasta","hay","haya","he","hecho","hemos","hicieron","hizo","hoy","hubo","igual","indico","informo","junto","lado","le","les","llego","lleva","llevar","luego","lugar","mas","manera","manifesto","mayor","me","mediante","mejor","menciono","menos","mi","misma","mismas","mismo","mismos","momento","mucha","muchas","mucho","nada","nadie","ni","ningun","ninguna","ningunas","ninguno","ningunos","no","nosotras","nuestra","nuestras","nuestro","nuestros","nueva","nuevas","nuevo","nuevos","nunca","o","ocho","otra","otras","otros","parece","parte","partir","pasada","pasado","pesar","poca","pocas","poco","pocos","podra","podran","podria","podrian","poner","posible","proximo","proximos","primer","primera","primeros","principalmente","propia","propias","propio","propios","pudo","pueda","pues","que","que","quedo","queremos","quien","quienes","quiere","realizo","realizado","realizar","respecto","si","solo","se","senalo","sea","sean","segun","segunda","segundo","seis","sera","seran","seria","sido","siempre","siete","sigue","siguiente","sino","sola","solas","solos","son","tal","tampoco","tan","tanto","tenia","tendra","tendran","tenga","tenido","tercera","toda","todas","todavia","todos","total","trata","traves","tres","tuvo","usted","varias","varios","veces","ver","vez","y","ya"})
stopwords_sp = list(stopwords_sp_web_accent_less)


In [ ]:
vect_tf = TfidfVectorizer(
    use_idf = False,            # default=True
    lowercase = True,           # Lowercase: default=True
    strip_accents = 'unicode',  # Remover acentos
    stop_words = stopwords_sp,  # Palabras para no tener en cuenta ya que no agregan mucho significado a una oracion.
    max_features = 500,         # maximo de vocabulario
    max_df = 0.8,               # (frecuencia máxima de documentos) es un valor que representa el porcentaje máximo de documentos en los que un término puede aparecer para ser incluido en el vocabulario. Los términos que aparecen en más del max_df% de los documentos se consideran comunes y, por lo tanto, no son relevantes para el análisis.
    min_df = 2,                 # (frecuencia mínima de documentos) es un valor entero que representa la frecuencia mínima con la que un término debe aparecer en los documentos para ser incluido en el vocabulario. Los términos que aparecen en menos de min_df documentos no se consideran relevantes para el análisis y se omiten del vocabulario.
    analyzer = 'word',
    #ngram_range=(2,6)
)

vect_tfidf = TfidfVectorizer(
    use_idf = True,             # default=True
    lowercase = True,           # Lowercase: default=True
    strip_accents = 'unicode',  # Remover acentos
    stop_words = stopwords_sp,  # Palabras para no tener en cuenta ya que no agregan mucho significado a una oracion.
    max_features = 500,         # maximo de vocabulario
    max_df = 0.8,               # (frecuencia máxima de documentos) es un valor que representa el porcentaje máximo de documentos en los que un término puede aparecer para ser incluido en el vocabulario. Los términos que aparecen en más del max_df% de los documentos se consideran comunes y, por lo tanto, no son relevantes para el análisis.
    min_df = 2,                 # (frecuencia mínima de documentos) es un valor entero que representa la frecuencia mínima con la que un término debe aparecer en los documentos para ser incluido en el vocabulario. Los términos que aparecen en menos de min_df documentos no se consideran relevantes para el análisis y se omiten del vocabulario.
    analyzer = 'word',
    #ngram_range=(2,6)
)

vect_binary = CountVectorizer(
    binary = True,              # default=True
    lowercase = True,           # Lowercase: default=True
    strip_accents = 'unicode',  # Remover acentos
    stop_words = stopwords_sp,  # Palabras para no tener en cuenta ya que no agregan mucho significado a una oracion.
    max_features = 500,         # maximo de vocabulario
    max_df = 0.8,               # (frecuencia máxima de documentos) es un valor que representa el porcentaje máximo de documentos en los que un término puede aparecer para ser incluido en el vocabulario. Los términos que aparecen en más del max_df% de los documentos se consideran comunes y, por lo tanto, no son relevantes para el análisis.
    min_df = 2,                 # (frecuencia mínima de documentos) es un valor entero que representa la frecuencia mínima con la que un término debe aparecer en los documentos para ser incluido en el vocabulario. Los términos que aparecen en menos de min_df documentos no se consideran relevantes para el análisis y se omiten del vocabulario.
    analyzer = 'word',
    #ngram_range=(2,6),
)

vect_count = CountVectorizer(
    binary = False,             # default=True
    lowercase = True,           # Lowercase: default=True
    strip_accents = 'unicode',  # Remover acentos
    stop_words = stopwords_sp,  # Palabras para no tener en cuenta ya que no agregan mucho significado a una oracion.
    max_features = 500,         # maximo de vocabulario
    max_df = 0.8,               # (frecuencia máxima de documentos) es un valor que representa el porcentaje máximo de documentos en los que un término puede aparecer para ser incluido en el vocabulario. Los términos que aparecen en más del max_df% de los documentos se consideran comunes y, por lo tanto, no son relevantes para el análisis.
    min_df = 2,                 # (frecuencia mínima de documentos) es un valor entero que representa la frecuencia mínima con la que un término debe aparecer en los documentos para ser incluido en el vocabulario. Los términos que aparecen en menos de min_df documentos no se consideran relevantes para el análisis y se omiten del vocabulario.
    analyzer = 'word',
    #ngram_range=(2,6),
)


In [ ]:
# Select the Representation

# 0: TF           # (float) TF Con normalización: Frecuencia de términos mide la frecuencia con la que aparece un término en un documento aplicando normalización.
# 1: TFIDF        # (float) TF por la IDF (Aumenta el peso de los terminos menos frecuentes y disminuye el peso de los terminos mas frecuentes)
# 2: BINARY       # (int) Coloca 1 o 0 si la palabra está contenida en el documento.
# 3: COUNT        # (int) TF Original: Coloca el total de palabras que aparecen en un documento.

representations = [ vect_tf, vect_tfidf, vect_binary, vect_count ]
represent_names = [ 'TF', 'TFIDF', 'BINARY', 'COUNT' ]

REP = 3

vect    = representations[REP]
X_train = vect.fit_transform(texts) # Return: sparse matrix of (n_samples, n_features) Tf-idf-weighted document-term matrix.


### Results

In [ ]:
# Classifiers

#SVM
svc = svm.SVC()
parameters_svc = [
    {'kernel': ['linear'], 'C':[ 10, 100, 1000, 100000], 'gamma': [0.001, 0.01, 0.1, 1 ]},
    {'kernel': ['rbf', 'poly', 'sigmoid'], 'gamma':[1e-3, 1e-4,'scale','auto'], 'C':[1, 10, 100, 1000, 100000]}
]

#LogisticRegression
lr = LogisticRegression(solver='liblinear', random_state=2)
parameters_lr = [{'solver':['sag', 'newton-cg',  'saga', 'liblinear'], 'max_iter':[1000]}]

#KNN
knn = Pipeline([('mms', MaxAbsScaler()), ('knn', KNeighborsClassifier())])
parameters_knn = [{'knn__n_neighbors': [1, 3, 5, 7, 8, 9, 10], 'knn__weights': ['uniform', 'distance'], 'knn__leaf_size': [15, 20]}]

#NaiveBayes Gaussian
gnb = GaussianNB()
parameters_gnb = [{'var_smoothing': [1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8, 1e-9, 1e-10, 1e-11, 1e-12, 1e-13, 1e-14, 1e-15]}]

#NaiveBayes Multinomial
gnm = MultinomialNB()
parameters_gnm = [{'alpha': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], 'fit_prior' : [True, False] }]

#RandomForest
rf = RandomForestClassifier() #Initialize with whatever parameters you want to
parameters_rf = [{'n_estimators': [5, 10, 15, 20], 'max_depth': [2, 5, 7, 9]}]

#DecisionTreeClassifier
dtc = DecisionTreeClassifier()
parameters_dtc = [{
    'splitter': ['best', 'random'], 'criterion': ["gini", "entropy"], "max_depth": [*range(1, 10)],
    'min_samples_leaf': [*range(1, 50, 5)],#' min_impurity_decrease': [*np.linspace(0, 0.5, 20)]
    #'max_features':[ None, 'auto','sqrt','log2'],   #Added
    #'min_impurity_decrease': [None, 0.5]            #Added
}]


# Group the Classifier with the corresponding params

classifier_and_params = [
  {'cslf':dtc , 'params':parameters_dtc, 'name':'dtc'},
  {'cslf':rf , 'params':parameters_rf, 'name':'rf'},
  {'cslf':knn , 'params':parameters_knn, 'name':'knn'},
  {'cslf':svc , 'params':parameters_svc, 'name':'svc'},
  {'cslf':gnb , 'params':parameters_gnb, 'name':'gnb'},
  {'cslf':gnm , 'params':parameters_gnm, 'name':'gnm'},
  {'cslf':lr , 'params':parameters_lr, 'name':'lr'}
]

#### GridSearch ####
# Para NB (NaiveBayes) se necesita tener X_traing.toarray(), ya que es este el tipo de valor que espera

def gridSearchProcess(classifier, params, name):
  clf = GridSearchCV(
      estimator=classifier,                               # Pass the model instance for which you want to check the hyperparameters.
      param_grid=params,                                  # the dictionary object that holds the hyperparameters you want to try
      scoring='f1_weighted',                              # evaluation metric that you want to use, you can simply pass a valid string/ object of evaluation metric
      cv=KFold(n_splits=5, shuffle=True, random_state=1), # number of cross-validation you have to try for each selected set of hyperparameters
      #refit=True,                                        # Refit an estimator using the best found parameters on the whole dataset (default=True).
      #n_jobs=-1,                                         # number of processes you wish to run in parallel for this task if it -1 it will use all available processors.
      #verbose=1,                                         # you can set it to 1 to get the detailed print out while you fit the data to GridSearchCV
  )
  clf.fit(X_train.toarray(), labels)

  # find best model score
  best_params       = clf.best_params_
  best_model_score  = clf.score(X_train.toarray(), labels)

  print(f'\n******** {name} Results ********')
  print(f'best_score:       {round(clf.best_score_, 4)}')
  print(f'scorer:           {clf.scoring}')
  print(f'best_params:      {clf.best_params_}')
  #print(f'best_model_score: {round(best_model_score, 4)}')
  #print(f'best_estimator:{clf.best_estimator_}')
  #print(f'best_index:       {clf.best_index_}')
  #print(f'X_traing:         {len(X_train.toarray())}')
  #print(f'n_splits:         {clf.n_splits_}')

print(f'Selected Tokenized Representation: {represent_names[REP]}')
print(f'Selected Class Personality: {personality[POS]}')

for pair in classifier_and_params:
    gridSearchProcess(classifier=pair['cslf'], params=pair['params'], name=pair['name'])

Selected Tokenized Representation: COUNT
Selected Class Personality: Responsabilidad

******** dtc Results ********
best_score:       0.8438
scorer:           f1_weighted
best_params:      {'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 31, 'splitter': 'best'}

******** rf Results ********
best_score:       0.8691
scorer:           f1_weighted
best_params:      {'max_depth': 9, 'n_estimators': 20}

******** knn Results ********
best_score:       0.6563
scorer:           f1_weighted
best_params:      {'knn__leaf_size': 15, 'knn__n_neighbors': 1, 'knn__weights': 'uniform'}

******** svc Results ********
best_score:       0.8797
scorer:           f1_weighted
best_params:      {'C': 1, 'gamma': 'scale', 'kernel': 'rbf'}

******** gnb Results ********
best_score:       0.8755
scorer:           f1_weighted
best_params:      {'var_smoothing': 0.001}

******** gnm Results ********
best_score:       0.8839
scorer:           f1_weighted
best_params:      {'alpha': 0.1, 'fit_prior': Tru